In [1]:
import netCDF4 as nc
from os import path

def add_disturbance(ncinfname, ncoutfname, toexclude):
    with nc.Dataset(ncinfname) as src, nc.Dataset(ncoutfname, "w") as dst:
        # copy global attributes all at once via dictionary
        dst.setncatts(src.__dict__)
        # copy dimensions
        for name, dimension in src.dimensions.items():
            dst.createDimension(
                name, (len(dimension) if not dimension.isunlimited() else None))
        # copy all file data except for the excluded
        for name, variable in src.variables.items():
            if name not in toexclude:
                x = dst.createVariable(name, variable.datatype, variable.dimensions)
                # copy variable attributes all at once via dictionary
                dst[name].setncatts(src[name].__dict__)
                dst[name][:] = src[name][:]

        # add disturbance
        dst_name = 'disturbance'
        x = dst.createVariable(dst_name, variable.datatype, variable.dimensions)
        attr = src['zeta_max'].__dict__
        attr['long_name'] = 'maximum water surface elevation departure above initial condition'
        attr['standard_name'] = 'disturbance'
        dst[dst_name].setncatts(attr)
        zeta_max = src['zeta_max'][:]
        depth = src['depth'][:]
        disturbance = [zeta_max[i] + min(depth[i], 0.0) for i in range(len(zeta_max))]
        dst[dst_name][:] = disturbance


basedir = "/projects/storm_surge/COMT/sbunya/runs/EmbedVEW1DtoLargeModel/NC/Florence"
ncfin = "maxele.63.nc"
ncfou = "maxele_plus_d.63.nc"

cases = ["SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5"]

# toexclude = ['depth', 'zeta_max', 'time_of_zeta_max']
toexclude = []

for case in cases:
    print(case)
    ncfname_in = path.join(basedir, case, 'maxele.63.nc')
    ncfname_ou = path.join(basedir, case, 'maxele_disturb.63.nc')
    add_disturbance(ncfname_in, ncfname_ou, toexclude)



SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5


/tmp/ipykernel_16612/988396461.py:30: UserWarning: Warning: converting a masked element to nan.
  dst[dst_name][:] = disturbance
